In [ ]:
```python
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extraire_contenu(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    return None

def extraire_titre(soup):
    return soup.find('h1', id='firstHeading').text.strip()

def extraire_sections_et_textes(soup):
    contenu = {}
    for section in soup.find_all(['h2', 'h3']):
        titre_section = section.text.strip().replace('[modifier | modifier le code]', '')
        paragraphes = []
        for sibling in section.find_next_siblings():
            if sibling.name in ['h2', 'h3']:
                break
            if sibling.name == 'p':
                paragraphes.append(sibling.text.strip())
        if paragraphes:
            contenu[titre_section] = "\n".join(paragraphes)
    return contenu

def extraire_liens_wikipedia(soup, base_url):
    liens = set()
    for lien in soup.find_all('a', href=True):
        href = lien['href']
        if href.startswith('/wiki/') and ':' not in href:
            liens.add(urljoin(base_url, href))
    return list(liens)

def extraire_wikipedia(url):
    html = extraire_contenu(url)
    if not html:
        return None
    soup = BeautifulSoup(html, 'html.parser')
    return {
        'title': extraire_titre(soup),
        'content': extraire_sections_et_textes(soup),
        'links': extraire_liens_wikipedia(soup, url)
    }

url_test = "https://fr.wikipedia.org/wiki/Python_(langage)"
resultat = extraire_wikipedia(url_test)

print("Titre de l'article :", resultat['title'])
print("\nSections de l'article :")
for section, texte in list(resultat['content'].items())[:3]:
    print(f"\n{section}:\n{texte[:200]}...")
print("\nLiens Wikipédia collectés :")
print(resultat['links'][:10])
```